<a href="https://colab.research.google.com/github/AlejandroVillazonG/Tareas_INF395/blob/main/T2/0_superclases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width= 800 /></center>
<hr style="height:2px;border:none"/>
<H1 align='center'> DESAFÍO TAREA 2</H1>

<H3 align='center'> INF395 2023-2 </H3>
<hr style="height:2px;border:none"/>

## Integrantes:
* Joaquín Aguirre (201910031-9)
* Alejandro Villazón (201910009-2)
* Dominique Yessouroun (201910005-K)

Importamos las librerías necesarias:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import os
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

import warnings
# Ignorar las advertencias
warnings.filterwarnings("ignore")

Definimos funciones auxiliares:

In [2]:
def preprocess_image(main_path, path_image, image_size):
    image = keras.utils.load_img(main_path + path_image)
    image = image.resize((image_size, image_size))
    return keras.utils.img_to_array(image, dtype='uint8')

def preprocess_train_test_set(main_path, paths, image_size):
    return np.array([preprocess_image(main_path, path_image, image_size) for path_image in paths])

In [3]:
def create_macro_f1(y_true, y_pred, num_classes):
    """
    Calcula el Macro F1-Score.
    """
    y_pred = tf.one_hot(tf.argmax(y_pred, axis=1), depth=num_classes)

    tp = tf.cast(tf.math.count_nonzero(y_pred * y_true, axis=1), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1-y_true), axis=1), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1-y_pred) * y_true, axis=1), tf.float32)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2*precision*recall / (precision + recall)
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return tf.reduce_mean(f1)

def transform_dishes(original_dishes, mapping_dishes, category_mapping):
    transformed_dishes = {}
    for original_key, dish in original_dishes.items():
        for category_key, category_value in category_mapping.items():
            if dish in mapping_dishes[category_value].values():
                # Encontrar la clave interna dentro del diccionario de categoría
                internal_key = [int(k) for k, v in mapping_dishes[category_value].items() if v == dish][0]
                # Convertir la clave original a int para guardarla en el resultado
                original_key_int = int(original_key)
                # Asegurar que la categoría exista en el diccionario resultante
                if category_key not in transformed_dishes:
                    transformed_dishes[category_key] = {}
                # Agregar el mapeo al diccionario resultante
                transformed_dishes[category_key][original_key_int] = internal_key
    return transformed_dishes

def create_model(image_size, num_classes):
  cnn_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
  for layer in cnn_model.layers:
      layer.trainable = False
  model = keras.Sequential([
      cnn_model,
      layers.GlobalAveragePooling2D(), #
      layers.Reshape((1, -1)), #
      layers.LSTM(256), #
      # layers.Conv2D(512, 2, activation='relu'),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(num_classes, activation='softmax')
  ])
  def macro_f1(y_true, y_pred):
    return create_macro_f1(y_true, y_pred, num_classes)

  model.compile(optimizer=keras.optimizers.AdamW(),
                loss=['categorical_crossentropy'],
                metrics=[macro_f1])
  return model

In [4]:
E_S = keras.callbacks.EarlyStopping(monitor='val_macro_f1', mode='max',
                                    patience=10, restore_best_weights=True)

Nos conectamos al drive:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/INF395/T2/'

## Preprocesar imágenes

In [7]:
!unzip '{path}inf-395-tarea-2.zip'  > /dev/null

In [8]:
main_path = '/content/'

df_train = pd.read_csv(main_path+'train.csv')

Definimos el tamaño que tendran las imágenes.

In [9]:
image_size = 225

In [10]:
%%time
X = preprocess_train_test_set(main_path, df_train['path'], image_size)

CPU times: user 5min 26s, sys: 14.5 s, total: 5min 40s
Wall time: 5min 39s


# NN para Superclase

In [11]:
df_train = pd.read_csv(path+'train.csv')
df_train[['food_category']].value_counts()

food_category
2                56250
1                15750
0                 3750
dtype: int64

In [12]:
index_0 = df_train[df_train['food_category'] == 0].sample(3750).index
index_1 = df_train[df_train['food_category'] == 1].sample(3750).index
index_2 = df_train[df_train['food_category'] == 2].sample(3750).index

X_balanced  = np.vstack([X[index_0], X[index_1], X[index_2]])
df_train_balanced = pd.concat([df_train.iloc[index_0],
                               df_train.iloc[index_1],
                               df_train.iloc[index_2]
                               ])
y = df_train_balanced['food_category']

In [13]:
y.value_counts()

0    3750
1    3750
2    3750
Name: food_category, dtype: int64

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_balanced, y, test_size=0.1, random_state=42, stratify=y)

In [15]:
y_val.value_counts()

2    375
0    375
1    375
Name: food_category, dtype: int64

## Entrenamiento de la NN para superclases

In [16]:
model_sup = create_model(image_size, 3)
model_sup.summary()

94765736/94765736 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 reshape (Reshape)           (None, 1, 2048)           0         
                                                                 
 lstm (LSTM)                 (None, 256)               2360320   
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32

In [17]:
y_train = tf.one_hot(y_train, depth=3)
y_val = tf.one_hot(y_val, depth=3)

In [18]:
history = model_sup.fit(X_train, y_train,
                    batch_size = 128,
                    epochs = 200,
                    callbacks=[E_S],
                    verbose = 1,
                    validation_data = (X_val, y_val)
                    )

Epoch 1/200
80/80 [==============================] - 57s 501ms/step - loss: 0.6846 - macro_f1: 0.6943 - val_loss: 0.6089 - val_macro_f1: 0.7405
Epoch 2/200
80/80 [==============================] - 37s 463ms/step - loss: 0.5379 - macro_f1: 0.7794 - val_loss: 0.5172 - val_macro_f1: 0.7840
Epoch 3/200
80/80 [==============================] - 38s 471ms/step - loss: 0.4847 - macro_f1: 0.7951 - val_loss: 0.4963 - val_macro_f1: 0.8044
Epoch 4/200
80/80 [==============================] - 37s 463ms/step - loss: 0.4429 - macro_f1: 0.8194 - val_loss: 0.5126 - val_macro_f1: 0.7940
Epoch 5/200
80/80 [==============================] - 38s 470ms/step - loss: 0.4057 - macro_f1: 0.8321 - val_loss: 0.5102 - val_macro_f1: 0.7923
Epoch 6/200
80/80 [==============================] - 37s 469ms/step - loss: 0.3715 - macro_f1: 0.8493 - val_loss: 0.4694 - val_macro_f1: 0.8153
Epoch 7/200
80/80 [==============================] - 37s 468ms/step - loss: 0.3445 - macro_f1: 0.8621 - val_loss: 0.5548 - val_macro_f1:

Guardamos la NN:

In [19]:
model_sup.save(path+'NNs/NN_superclases.keras')